In [2]:
import numpy as np
import torch
from torch import nn
import os, random, math
from tqdm import tqdm
import matplotlib.pyplot as plt
from lstm_class import LSTM_Single_Layer            # *!*!*!*!* ensure correct LSTM structure is loaded !*!**!*!*!
from path_functions import check_if_avoids_obstacles

In [ ]:
# CONSTANTS:
# for loading encoder and lstm:
LOAD_PATH_ENC = "checkpoints/encoder/"          # path to where encoder is
ENC_NAME = "pretty_decent_encoder"              # name of saved encoder
checkpoint_enc = torch.load(f'{LOAD_PATH_ENC}{ENC_NAME}.tar')

LOAD_PATH_LSTM = "checkpoints/lstm/"            # path to where lstm is
LSTM_NAME = "trained_on_60000_paths"            # name of saved lstm
checkpoint_lstm = torch.load(f'{LOAD_PATH_LSTM}{LSTM_NAME}.tar')

# for loading dataset:
DATASET = 'random_20_density/'
MAP_SHAPE = (64,64)
NUM_TRAIN_PATHS = 10
NUM_EVAL_PATHS = 10
NUM_GENERATED_POINTS = 30

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class PathsDataset(torch.utils.data.Dataset):
    # init the dataset, shape = L x W
    def __init__(self, gen_dir, transform=None, shape = (100,100), device='cpu', stop_after=500):
        self.device = device
        self.maps_and_paths = []

        num_maps = 0
        for directory in tqdm(os.listdir(gen_dir)):
            num_maps += 1
            dir_path = os.path.join(gen_dir, directory)
            sequence = []

            # open map file:
            with open(f"{dir_path}/{directory}.txt", 'r') as f:
                self.flat_map = np.loadtxt(f)
                self.map = np.asarray(self.flat_map, dtype=np.float32).reshape(shape[0],shape[1])

            # add map to sequence array:
            sequence.append(torch.tensor(self.map))

            path_dir = f"{dir_path}/paths/"
            num_paths = 0
            for filename in os.listdir(path_dir):
                num_paths += 1
                with open(os.path.join(path_dir, filename), 'r') as f: # open in readonly mode
                    path_points_list = [] # a list to hold each point in a path
                    self.flat_path = np.loadtxt(f) # load in the flat path from file
                    self.path = np.asarray(self.flat_path, dtype=np.float32).reshape(len(self.flat_path)//2,2) #unflatten the path from the file
                    for point in self.path:
                        x = point[0]
                        y = point[1]
                        this_point = [x, y]
                        path_points_list.append(this_point)
                        
                sequence.append(torch.tensor(path_points_list, dtype=torch.float)[:, :])
                
            self.maps_and_paths.append(sequence)

            if num_maps == stop_after:
                break
        
        self.transform = transform
        print("Done!")

    def __len__(self):
        return len(self.maps_and_paths)

    def __getitem__(self, idx):
        x = self.maps_and_paths[idx]
        if self.transform:
            x = self.transform(x)
        
        return x

In [ ]:
# DEFINE ENCODER STRUCTURE:
class Encoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 16, 5, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 5, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 5, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),     
            nn.Conv2d(64, 128, 7, stride=2, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),  
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        
    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        return x

In [ ]:
# CREATE ENCODER INSTANCE:
encoder = Encoder()

# CREATE LSTM INSTANCE:
rnn = LSTM_Single_Layer(device=device, dropout=checkpoint_lstm['dropout'], dropout_p=checkpoint_lstm['dropout_p'])

In [ ]:
# LOAD ENCODER:
encoder.load_state_dict(checkpoint_enc['model_state_dict'])
encoder.to(device)
encoder.eval()

# LOAD LSTM:
rnn.load_state_dict(checkpoint_lstm['model_state_dict'])
rnn.to(device)
rnn.eval()

In [ ]:
# LOAD PATHS:
train_dataset = PathsDataset(gen_dir = f"./env/{DATASET}/train", shape = MAP_SHAPE, transform=None, device=device, stop_after=1000)
eval_dataset = PathsDataset(gen_dir = f"./env/{DATASET}/eval", shape = MAP_SHAPE, transform=None, device=device, stop_after=1000)
train_len = len(train_dataset)
eval_len = len(eval_dataset)

In [ ]:
# GENERATING LSTM PATHS FROM TRAIN DATA:

generated_train_paths = []
random_train_vals = []
valid_train_paths = 0

for i in range(NUM_TRAIN_PATHS):
    # choose random map:
    map_num = random.randint(0, train_len-1)
    map = train_dataset[map_num][0].to(device)
    encoded_map = encoder(map.unsqueeze(0).unsqueeze(0))

    # choose random path:
    path_num = random.randint(1, len(train_dataset[map_num])-1)
    path = train_dataset[map_num][path_num]

    # generate lstm path:
    start_point = path[0].to(device)
    goal_point = path[len(path)-1].to(device)
    generated_pts = []
    with torch.no_grad():
        # add start point:
        generated_pts.append(start_point.cpu().detach().numpy().tolist())
        prev_pred = start_point

        # predict and add other points:
        for i in range(NUM_GENERATED_POINTS):
            pred = rnn(goal_point, prev_pred, encoded_map).squeeze()
            prev_pred = pred
            generated_pts.append(pred.cpu().detach().numpy().tolist())

    generated_train_paths.append(generated_pts)
    random_train_vals.append([map_num, path_num])

    # checking if valid path:
    xs = []
    ys = []

    for x,y in generated_pts:
        xs.append(x)
        ys.append(y)

    valid_train_paths += check_if_avoids_obstacles(xs, ys, map.cpu().detach().numpy(), check_dis=0.5)

In [ ]:
# CREATING FIGURE FOR TRAIN DATA:

print(f"Percentage of successful paths: {valid_train_paths/NUM_TRAIN_PATHS*100}%")
print(f"Num of maps: {len(train_dataset)}")
print(f"Total num of paths: {len(train_dataset)*(len(train_dataset[0])-1)}")

fig = plt.figure(figsize=(15,30))
fig.set_facecolor('white')

col_title = True
for i in range(NUM_TRAIN_PATHS):

    map_idx = random_train_vals[i][0]
    path_idx = random_train_vals[i][1]

    map = train_dataset[map_idx][0]
    start_point = train_dataset[map_idx][path_idx][0]
    goal_point = train_dataset[map_idx][path_idx][len(train_dataset[map_idx][path_idx])-1]

    xs = []
    ys = []
    for x,y in generated_train_paths[i]:
        xs.append(x)
        ys.append(y)

    sub = fig.add_subplot(NUM_TRAIN_PATHS,4,i*4+1)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    if col_title:
        plt.title('Obstacles')

    sub = fig.add_subplot(NUM_TRAIN_PATHS,4,i*4+2)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    # plot truth path
    plt.plot(train_dataset[map_idx][path_idx][:,0], train_dataset[map_idx][path_idx][:,1], color='orange')
    plt.scatter(goal_point[0], goal_point[1], color="r")
    plt.scatter(start_point[0], start_point[1], color="g")
    if col_title:
        plt.title('RRT Star Paths')

    sub = fig.add_subplot(NUM_TRAIN_PATHS,4,i*4+3)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    plt.plot(xs, ys, color='aqua', label = "predicted")
    plt.scatter(goal_point[0], goal_point[1], color="r")
    plt.scatter(start_point[0], start_point[1], color="g")
    if col_title:
        plt.title('LSTM Paths')

    col_title = False

plt.savefig("train_output.png", facecolor=fig.get_facecolor(), bbox_inches='tight')
plt.show()

In [ ]:
# GENERATING LSTM PATHS FROM EVAL DATA:

generated_eval_paths = []
random_eval_vals = []
valid_eval_paths = 0

for i in range(NUM_EVAL_PATHS):
    # choose random map:
    map_num = random.randint(0, eval_len-1)
    map = eval_dataset[map_num][0].to(device)
    encoded_map = encoder(map.unsqueeze(0).unsqueeze(0))

    # choose random path:
    path_num = random.randint(1, len(eval_dataset[map_num])-1)
    path = eval_dataset[map_num][path_num]

    # generate lstm path:
    start_point = path[0].to(device)
    goal_point = path[len(path)-1].to(device)
    generated_pts = []
    with torch.no_grad():
        # add start point:
        generated_pts.append(start_point.cpu().detach().numpy().tolist())
        prev_pred = start_point

        # predict and add other points:
        for i in range(NUM_GENERATED_POINTS):
            pred = rnn(goal_point, prev_pred, encoded_map).squeeze()
            prev_pred = pred
            generated_pts.append(pred.cpu().detach().numpy().tolist())

    generated_eval_paths.append(generated_pts)
    random_eval_vals.append([map_num, path_num])

    # checking if valid path:
    xs = []
    ys = []

    for x,y in generated_pts:
        xs.append(x)
        ys.append(y)

    valid_eval_paths += check_if_avoids_obstacles(xs, ys, map.cpu().detach().numpy(), check_dis=0.5)

In [ ]:
# CREATING FIGURE FOR EVAL DATA:

print(f"Percentage of successful paths: {valid_eval_paths/NUM_EVAL_PATHS*100}%")
print(f"Num of maps: {len(train_dataset)}")
print(f"Total num of paths: {len(train_dataset)*(len(train_dataset[0])-1)}")

fig = plt.figure(figsize=(15,30))
fig.set_facecolor('white')

col_title = True
for i in range(NUM_EVAL_PATHS):

    map_idx = random_eval_vals[i][0]
    path_idx = random_eval_vals[i][1]

    map = eval_dataset[map_idx][0]
    start_point = eval_dataset[map_idx][path_idx][0]
    goal_point = eval_dataset[map_idx][path_idx][len(eval_dataset[map_idx][path_idx])-1]

    xs = []
    ys = []
    for x,y in generated_eval_paths[i]:
        xs.append(x)
        ys.append(y)

    sub = fig.add_subplot(NUM_EVAL_PATHS,4,i*4+1)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    if col_title:
        plt.title('Obstacles')

    sub = fig.add_subplot(NUM_EVAL_PATHS,4,i*4+2)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    # plot truth path
    plt.plot(eval_dataset[map_idx][path_idx][:,0], eval_dataset[map_idx][path_idx][:,1], color='orange')
    plt.scatter(goal_point[0], goal_point[1], color="r")
    plt.scatter(start_point[0], start_point[1], color="g")
    if col_title:
        plt.title('RRT Star Paths')

    sub = fig.add_subplot(NUM_EVAL_PATHS,4,i*4+3)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(map.cpu())
    plt.plot(xs, ys, color='aqua', label = "predicted")
    plt.scatter(goal_point[0], goal_point[1], color="r")
    plt.scatter(start_point[0], start_point[1], color="g")
    if col_title:
        plt.title('LSTM Paths')

    col_title = False

plt.savefig("eval_output.png", facecolor=fig.get_facecolor(), bbox_inches='tight')
plt.show()